In [1]:
# import yfinance as yf
# import feedparser
from dotenv import load_dotenv
import pandas as pd
import multiextractor
# from bs4 import BeautifulSoup
# import requests
# import tldextract
from urllib.parse import urlparse
# from datetime import datetime, date, time
import spacy
# import asyncio
from concurrent.futures import ProcessPoolExecutor
from test_extract_rss import create_entry_from_rss
# import multiprocessing

load_dotenv()

nlp = spacy.load('en_core_web_md')

In [2]:
# r = requests.get('https://www.cnbc.com/2023/09/27/amazon-lawsuit-protects-free-and-fair-competition-ftcs-lina-khan.html')
# soup = BeautifulSoup(r.content)
# # b_list = soup.find('div', {'class': 'ArticleBody-articleBody'}).find('div', {'class': 'group'}).find_all('p')
# # b_para = ' '.join([i.text for i in b_list])

# b_art_body = multiextractor.locate_elements(soup, 'div', 'find', attrs={'class': 'ArticleBody-articleBody'})
# b_list = multiextractor.process_text(b_art_body, 'summary', 'find', 'div', attrs={'class': 'group'})
# b_para = ' '.join(b_list)

In [3]:
# # async version - need to try
# def extract_rss_body(url, body_attrs=None, list_attrs=None) -> str:
#     _body_kwargs, _list_kwargs = {}, {}
#     r = requests.get(url)
#     soup = BeautifulSoup(r.content)
    
#     if body_attrs is not None: _body_kwargs.update({'attrs': body_attrs})
#     _art_body = multiextractor.locate_elements(soup, 'div', 'find', **_body_kwargs)
    
#     if list_attrs is not None: _list_kwargs.update({'attrs': list_attrs})
#     _body_list = multiextractor.process_text(_art_body, 'summary', 'find', 'div', **_list_kwargs)
#     return ' '.join(_body_list)

# async def create_entry_from_rss(url, executor, body_attrs=None, list_attrs=None) -> pd.DataFrame:
#     _feeds = feedparser.parse(url)
#     _entry_list = []
#     loop = asyncio.get_event_loop()
#     for _entry in _feeds.entries:
#         _entry.body = await loop.run_in_executor(
#             executor,
#             extract_rss_body,
#             _entry.link, 
#             body_attrs, 
#             list_attrs
#         ) if (body_attrs is not None) & (list_attrs is not None) else _entry.summary
#         _tmp = {
#             'title': _entry.title,
#             'description': _entry.summary,
#             'content': _entry.body,
#             'url': _entry.link,
#             'image': '',
#             'publishedAt': _entry.published,
#             'name': tldextract.extract(_entry.link).domain.title(),
#             'domainName': urlparse(_entry.link).netloc,
#             'publishedDate': date(_entry.published_parsed.tm_year, _entry.published_parsed.tm_mon, _entry.published_parsed.tm_mday),
#             'publishedTime': time(_entry.published_parsed.tm_hour, _entry.published_parsed.tm_min, _entry.published_parsed.tm_sec)
#         }
#         _entry_list.append(_tmp)
#     _df = pd.DataFrame(_entry_list)
#     _df['publishedAt'] = pd.to_datetime(_df['publishedAt']).dt.strftime('%Y%m%d %H:%M:%S%z+00:00')
#     return _df

In [4]:
# def extract_rss_body(url, body_attrs=None, list_attrs=None) -> str:
#     _body_kwargs, _list_kwargs = {}, {}
#     r = requests.get(url)
#     soup = BeautifulSoup(r.content)
    
#     if body_attrs is not None: _body_kwargs.update({'attrs': body_attrs})
#     _art_body = multiextractor.locate_elements(soup, 'div', 'find', **_body_kwargs)
    
#     if list_attrs is not None: _list_kwargs.update({'attrs': list_attrs})
#     _body_list = multiextractor.process_text(_art_body, 'summary', 'find', 'div', **_list_kwargs)
#     return ' '.join(_body_list)

# def create_entry_from_rss(url, body_attrs=None, list_attrs=None) -> pd.DataFrame:
#     _feeds = feedparser.parse(url)
#     _entry_list = []
#     for _entry in _feeds.entries:
#         _entry.body = extract_rss_body(_entry.link, body_attrs=body_attrs, list_attrs=list_attrs) if (body_attrs is not None) & (list_attrs is not None) else _entry.summary
#         _tmp = {
#             'title': _entry.title,
#             'description': _entry.summary,
#             'content': _entry.body,
#             'url': _entry.link,
#             'image': '',
#             'publishedAt': _entry.published,
#             'name': tldextract.extract(_entry.link).domain.title(),
#             'domainName': urlparse(_entry.link).netloc,
#             'publishedDate': date(_entry.published_parsed.tm_year, _entry.published_parsed.tm_mon, _entry.published_parsed.tm_mday),
#             'publishedTime': time(_entry.published_parsed.tm_hour, _entry.published_parsed.tm_min, _entry.published_parsed.tm_sec)
#         }
#         _entry_list.append(_tmp)
#     _df = pd.DataFrame(_entry_list)
#     _df['publishedAt'] = pd.to_datetime(_df['publishedAt']).dt.strftime('%Y%m%d %H:%M:%S%z+00:00')
#     return _df

In [5]:
# cnbc
executor = ProcessPoolExecutor(8)

url = 'https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=19854910'
# cnbc_arts = create_entry_from_rss(url, body_attrs={'class': 'ArticleBody-articleBody'}, list_attrs={'class': 'group'})
cnbc_arts = await create_entry_from_rss(url, executor, body_attrs={'class': 'ArticleBody-articleBody'}, list_attrs={'class': 'group'})

In [6]:
test1 = cnbc_arts.copy()
test1 = multiextractor.process_datetime(test1)
test1 = multiextractor.process_sentence_count(test1, nlp, 'title', 'description', 'content')
test1 = multiextractor.process_token_count(test1, nlp, 'title', 'description', 'content')

In [7]:
# trading economics
executor = ProcessPoolExecutor(8)

url = 'https://tradingeconomics.com/rss/news.aspx?i=bank+lending+rate'
# trade_econ_blr = create_entry_from_rss(url)
trade_econ_blr = await create_entry_from_rss(url, executor)

In [8]:
test2 = trade_econ_blr.copy()
test2 = multiextractor.process_datetime(test2)
test2 = multiextractor.process_sentence_count(test2, nlp, 'title', 'description', 'content')
test2 = multiextractor.process_token_count(test2, nlp, 'title', 'description', 'content')

In [9]:
# # cnbc
# url = 'https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=19854910'
# feed = feedparser.parse(url)
# for entry in feed.entries:
#     entry.title
#     entry.title_detail.language
#     entry.link
#     entry.summary
#     entry.summary_detail.language
#     entry.published
#     pub_year = entry.published_parsed.tm_year
#     pub_mth = entry.published_parsed.tm_mon
#     pub_day = entry.published_parsed.tm_mday
#     pub_hr = entry.published_parsed.tm_hour
#     pub_min = entry.published_parsed.tm_min
#     pub_sec = entry.published_parsed.tm_sec
#     pub_wkday = entry.published_parsed.tm_wday

In [10]:
# # tradingeconomics
# url = 'https://tradingeconomics.com/rss/news.aspx?i=bank+lending+rate'
# feed = feedparser.parse(url)
# for entry in feed.entries:
#     entry.title
#     entry.title_detail.language
#     entry.link
#     entry.summary
#     entry.published
#     pub_year = entry.published_parsed.tm_year
#     pub_mth = entry.published_parsed.tm_mon
#     pub_day = entry.published_parsed.tm_mday
#     pub_hr = entry.published_parsed.tm_hour
#     pub_min = entry.published_parsed.tm_min
#     pub_sec = entry.published_parsed.tm_sec
#     pub_wkday = entry.published_parsed.tm_wday

In [23]:
conn_params = multiextractor.DBConstLoader('cnbc')

In [37]:
def sql_insert_articles(df: pd.DataFrame, conn_params: multiextractor.DBConstLoader, table_name: str, constraint_col: str | None):
    '''
    Creates and executes SQL script for article insertion
    
    :params:
    df: DataFrame object - table containing data to be inserted
    constraint_col: str - column name indicating key column upon detecting duplicates
    '''
    conn = multiextractor.pg_connection(conn_params)
    cur = conn.cursor()
    
    sql_col_str = ''.join(['("', '","'.join(df.columns), '")'])
    sql_val_str = ''.join(['(', ','.join(['%s'] * df.shape[1]), ')'])
    args_str = ','.join(cur.mogrify(sql_val_str, x).decode('utf-8') for x in df.values)
    
    if constraint_col is None:
        insert_script = f'INSERT INTO {table_name} {sql_col_str} VALUES {args_str}'
    else:
        sql_col_ups_str = ''.join(['(EXCLUDED."', '", EXCLUDED."'.join(df.columns), '")'])
        update_query_template = '{} = {}'.format(sql_col_str, sql_col_ups_str)
        insert_script = f'''
            INSERT INTO {table_name} {sql_col_str}
            VALUES {args_str}
            ON CONFLICT ("{constraint_col}") DO
            UPDATE SET {update_query_template}
        '''
    # print(insert_script)
    try:
        cur.execute(insert_script)
        conn.commit()
        conn.close()
        print('All articles inserted')
    except Exception as e:
        print(e)
        print('Error encountered in article insertion process')

def sql_create_table(df: pd.DataFrame, conn_params: multiextractor.DBConstLoader, table_name: str = 'articles', unique_col: str | None = None):
    '''
    Creates and executes SQL script for article table creation
    
    :params:
    df: DataFrame object - data table to extract table schema for SQL table creation query
    table_name: str - name of table to be created in Postgres database
    unique_col: str - name of column to set as unique index in created table 
    '''
    table_script = pd.io.sql.get_schema(df, table_name)
    idx = table_script.index('TABLE')
    table_script = table_script[:idx + len('TABLE')] + ' IF NOT EXISTS' + table_script[idx + len('TABLE'):]

    if unique_col is not None:
        # to_replace_str = '"' + unique_col + '"'
        to_replace_str = ','
        idx = table_script.index(to_replace_str)
        # len_replace_str = len(to_replace_str)
        # table_script = table_script[: (idx + len_replace_str)] + ' UNIQUE' + table_script[(idx + len_replace_str):]
        table_script = table_script[:idx] + ' UNIQUE' + table_script[idx:]

    # print(table_script)
    with multiextractor.pg_connection(conn_params) as conn:
        try:
            cur = conn.cursor()
            cur.execute(table_script)
        except Exception as e:
            print(e)
            print('Error encountered in table creation process')

In [38]:
# sql_create_table(cnbc_arts, conn_params, table_name='cnbc_articles', unique_col='title')
sql_insert_articles(cnbc_arts, conn_params, table_name='cnbc_articles', constraint_col='title')

TypeError: sql_insert_articles() missing 1 required positional argument: 'table_name'

In [ ]:
import os
import psycopg2

USER = os.getenv('COCKROACH_USER')
PW = os.getenv('COCKROACH_PW')
HOST = os.getenv('COCKROACH_HOST')
PORT = os.getenv('COCKROACH_PORT')
DB = os.getenv('COCKROACH_DB')
conn_str = f'postgresql://{USER}:{PW}@{HOST}:{PORT}/{DB}'

# conn = psycopg2.connect(conn_str)
# with conn.cursor() as cur:
#     cur.execute("SELECT now()")
#     res = cur.fetchall()
#     conn.commit()
#     print(res)